## Lissajous Curves with Python FuncAnimation

This project uses the Python FuncAnimation library to animate and customize parametric functions and differential equations, including Lissajous Curves and the Duffing Oscillator.

In [21]:
# Imports
import matplotlib; matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from scipy.integrate import odeint

The first class, *AnimationParameters*, is used to define revelant parameters used for plotting and the animation call. The parameters include **interval**, **frames**, **trail length**, **color**, **marker**, **marker size**, **line width**, and parametric functions for a 2D axes. 

In [22]:
class AnimationParameters:
    def __init__(self, interval, frames, trail_length, color, marker, marker_size, line_width, x_parametric, y_parametric):
        self.interval = interval
        self.frames = frames
        self.trail_length = trail_length
        self.color = color
        self.marker = marker
        self.marker_size = marker_size
        self.line_width = line_width
        self.x_parametric = x_parametric
        self.y_parametric = y_parametric

    def set_parameters(self, interval2, frames2, trail_length2, color2, marker2, marker_size2, line_width2, x_parametric2, y_parametric2):
        self.interval = interval2
        self.frames = frames2
        self.trail_length = trail_length2
        self.color = color2
        self.marker = marker2
        self.markersize = marker_size2
        self.linewidth = line_width2
        self.x_parametric = x_parametric2
        self.y_parametric = y_parametric2

    def get_parameters(self):
        return [self.interval, self.frames, self.trail_length, self.color, self.marker,
        self.marker_size, self.line_width, self.x_parametric, self.y_parametric]

The next class, *AnimationPlotter*, uses parameters from *AnimationParameters* to create a plot, trail, and animation. The **plot** method creates all necessary plot attributes and the trail object. The **update_trail** method updates the trail array at each frame by evaluating its parametrized coordinates and ensures the array does not exceed the trail length. The **animation** method will then create the animation. 

In [23]:
class AnimationPlotter:
    def __init__(self, animation_parameter_object):
        self.animation_parameter_object = animation_parameter_object
        self.x_trail = []
        self.y_trail = []
        self.fig = None
        self.ax = None
        #self.trail, =  None

    def plot(self):
        self.fig, self.ax = plt.subplots(dpi=500)
        self.ax.axis([-15,15,-15,15])
        self.ax.set_aspect("equal")
        self.ax.set_facecolor('xkcd:black')
        self.ax.tick_params(axis='x', bottom=False, labelbottom=False)
        self.ax.tick_params(axis='y', left=False, labelleft=False)
        self.trail, = self.ax.plot(0, 0, alpha=1, linewidth = self.animation_parameter_object[6], color = self.animation_parameter_object[3], marker = self.animation_parameter_object[4], markersize = self.animation_parameter_object[5]) 
    
    def update_trail(self, phi): 
        x_func = self.animation_parameter_object[7]
        y_func = self.animation_parameter_object[8]
        x = x_func(phi)
        y = y_func(phi)

        self.x_trail.append(x) 
        self.y_trail.append(y)

        if len(self.x_trail) > self.animation_parameter_object[2]:
            self.x_trail.pop(0)
            self.y_trail.pop(0)

        self.trail.set_data(self.x_trail, self.y_trail)
        return self.trail, 

    def animation(self): 
        ani = FuncAnimation(self.fig, self.update_trail, interval=self.animation_parameter_object[0], blit=True, repeat=True,
                    frames=self.animation_parameter_object[1])
        plt.show()

Finally, objects are initialized to create the animations.

In [24]:
# Parametric equations
def x_parametric(phi):
    return 12 * np.sin(2* phi + phi)

def y_parametric(phi):
    return 7 * np.sin(16 * phi)

# Class calls
parameter_object = AnimationParameters(30, np.linspace(0, 3*np.pi, 360*4), 100, "lime", "o", 0.5, 0, x_parametric, y_parametric)
retrieve_parameter_object = parameter_object.get_parameters() 

plot_object = AnimationPlotter(retrieve_parameter_object)
plot_object_initialize = plot_object.plot()
plot_object_animate = plot_object.animation()

 The *DifferentialEquationParameters* class solves differential equations and returns two solution arrays. Then, the *AnimationPlotterODE* class will return animated renders of the differential equation. Note *AnimationPlotterODE* is adapted from the original *AnimationPlotter* as it's working with arrays and not parametric equations. In future these classes may be refined to handle both inputs and additonal parameters.

In [25]:
class AnimationPlotterODE:
    def __init__(self, animation_parameter_object, ODEarrays): 
        self.animation_parameter_object = animation_parameter_object
        self.x_trail = []
        self.y_trail = []
        self.ODEarrays = ODEarrays
        self.fig = None
        self.ax = None
        #self.trail, =  None

    def plot(self):
        self.fig, self.ax = plt.subplots(dpi=500)
        self.ax.axis([-15,15,-15,15])
        self.ax.set_aspect("equal")
        self.ax.set_facecolor('xkcd:black')
        self.ax.tick_params(axis='x', bottom=False, labelbottom=False)
        self.ax.tick_params(axis='y', left=False, labelleft=False)
        self.trail, = self.ax.plot(0, 0, alpha=1, linewidth = self.animation_parameter_object[6], color = self.animation_parameter_object[3], marker = self.animation_parameter_object[4], markersize = self.animation_parameter_object[5]) 
    
    def update_trail(self, phi): 
        x = self.ODEarrays[0][phi]
        y = self.ODEarrays[1][phi]

        self.x_trail.append(x) 
        self.y_trail.append(y)

        if len(self.x_trail) > self.animation_parameter_object[2]:
            self.x_trail.pop(0)
            self.y_trail.pop(0)

        self.trail.set_data(self.x_trail, self.y_trail)
        return self.trail, 

    def animation(self): 
        ani = FuncAnimation(self.fig, self.update_trail, interval=self.animation_parameter_object[0], blit=True, repeat=True,
                    frames=self.animation_parameter_object[1])
        plt.show()

In [26]:
class DifferentialEquationParameters:
    def __init__(self, time_values, initial_position, initial_velocity, scale_factor):
        self.time_values = time_values
        self.initial_position = initial_position 
        self.initial_velocity = initial_velocity
        self.scale_factor = scale_factor
    
    def DifferentialEquationSolver(self, myODE):
        solution = self.scale_factor * odeint(myODE, [self.initial_position, self.initial_velocity] , self.time_values)
        position2 = solution[:, 0]
        velocity2 = solution[:, 1] 
        return position2, velocity2

In [27]:
# Differential Equation
def Duffing_ODE(y, t):
    delta, alpha, beta, gamma, omega = 0.2, -1, 1, 0.3, 1.2
    x, v = y
    dxdt = v
    dvdt = - delta * v - alpha * x - beta * (x ** 3) + gamma * np.cos(omega * t)
    return [dxdt, dvdt]

# Parameters
times = times = np.linspace(0, 10, 100)
initial_conditions = [0.1, 0.0]
scale_factor = 2

solution = scale_factor * odeint(Duffing_ODE, initial_conditions, times)
position2 = solution[:, 0]
velocity2 = solution[:, 1] 


# Class calls
differential_equation_parameters_object = DifferentialEquationParameters(np.linspace(0, 10, 100), 0.1, 0.0, 6)
differential_equation_solver_object = differential_equation_parameters_object.DifferentialEquationSolver(Duffing_ODE)

parameter_object_2 = AnimationParameters(30, 100, 100, "lime", "o", 0.5, 0, 0, 0)
retrieve_parameter_object_2 = parameter_object_2.get_parameters() 

plot_object_2 = AnimationPlotterODE(retrieve_parameter_object_2, differential_equation_solver_object)
plot_object_initialize_2 = plot_object_2.plot()
plot_object_animate_2 = plot_object_2.animation()